In [16]:
from xhtml2pdf import pisa
from xhtml2pdf.config.httpconfig import httpConfig

In [2]:
def _convert_html2pdf(source_html, output_path):
    # TODO: make parent directory
    # TODO: check file extension
    httpConfig.save_keys('nosslcheck', True)
    with open(output_path, "w+b") as f:
        pisa_status = pisa.CreatePDF(source_html, dest=f)
        
    return pisa_status.err

In [3]:
table_contents_css = """
pdftoc {
    color: #666;
}
pdftoc.pdftoclevel0 {
    font-weight: bold;
    margin-top: 0.5em;
}
pdftoc.pdftoclevel1 {
    margin-left: 1em;
}
pdftoc.pdftoclevel2 {
    margin-left: 2em;
    font-style: italic;
}"""

In [4]:
inline_block_template = """
.box{
 display: inline-block;
 width: 200px;
}

"""

In [5]:
# State

In [7]:
import pickle
import time
from typing import List
import datetime as dt
from pathlib import Path

import numpy as np

import plotly.graph_objects as go
import plotly.express as px

from tqdm import tqdm

from quara.data_analysis import data_analysis, physicality_violation_check, report
from quara.objects.composite_system import CompositeSystem
from quara.objects.elemental_system import ElementalSystem
from quara.objects.matrix_basis import get_normalized_pauli_basis
from quara.objects.povm import (
    Povm,
    get_x_measurement,
    get_y_measurement,
    get_z_measurement,
)
from quara.objects.qoperation import QOperation
from quara.objects.state import State, get_z0_1q, get_z1_1q, get_x0_1q
from quara.protocol.qtomography.standard.standard_qst import StandardQst
from quara.protocol.qtomography.standard.linear_estimator import LinearEstimator
from quara.protocol.qtomography.standard.projected_linear_estimator import (
    ProjectedLinearEstimator,
)

In [8]:
%load_ext autoreload
%autoreload 2

In [9]:
# setup system
e_sys = ElementalSystem(0, get_normalized_pauli_basis())
c_sys = CompositeSystem([e_sys])

povm_x = get_x_measurement(c_sys)
povm_y = get_y_measurement(c_sys)
povm_z = get_z_measurement(c_sys)
povms = [povm_x, povm_y, povm_z]

In [10]:
true_object = get_z0_1q(c_sys)
num_data = [100, 1000]
iteration = 100

# Linear (on_para_eq_constraint=True)
qst = StandardQst(povms, on_para_eq_constraint=True)
estimation_results_affine = data_analysis.estimate(qtomography=qst,
                                                                         true_object=true_object,
                                                                         num_data=num_data,
                                                                         estimator=LinearEstimator(),
                                                                         iteration=iteration)

# Linear (on_para_eq_constraint=False)
qst = StandardQst(povms, on_para_eq_constraint=False)
estimation_results_linear = data_analysis.estimate(qtomography=qst,
                                                                         true_object=true_object,
                                                                         num_data=num_data,
                                                                         estimator=LinearEstimator(),
                                                                         iteration=iteration)

100%|██████████| 100/100 [00:01<00:00, 78.23it/s]


一時保存時のファイル名規則:
{ケース名}_{グラフの種類}_{numの情報}_{グラフの種類固有の識別子}

例: 
等式制約の場合:    
- case=0_trace_num=100
不等式制約の場合
- case=0_eigenvalues_num=100_i=0
- case=0_sum-unphysical-eigenvalues_num=100_greater-than-one
- case=0_sum-unphysical-eigenvalues_num=100_less-than-zero

In [36]:
# MSE
target_list = [estimation_results_affine, estimation_results_linear]
case_name_list = ["LinearEstimator(True)", "LinearEstimator(False)"]

iteration = 100

mse_div = report.generate_mse_div(
    target_list, case_name_list, true_object, num_data, iteration, [qst]
)

In [359]:
# 残すこと
case_id = 0
case_name = "LinearEstimator(True)"
target = estimation_results_affine

physicality_violation_test_div_true = f"""
<h2>on_para_eq_constraint=True</h2>
<h3>case {case_id}: {case_name}<h3>
{report.generate_trace_div(target, case_id=case_id, num_data=num_data)}
"""

# Falseの場合
case_id = 1
case_name = "LinearEstimator(False)"
target = estimation_results_linear

physicality_violation_test_div_false = f"""
<h2>on_para_eq_constraint=False</h2>
<h3>case {case_id}: {case_name}<h3>
<h4>Eigenvalue</h4>
{report.generate_eigenvalues_div(target, case_id=case_id, num_data=num_data, true_object=true_object)}
<h4>Sum of unphysical eigenvalues </h4>
{report.generate_sum_eigenvalues_div(target, case_id=case_id, num_data=num_data, true_object=true_object)}
"""

physicality_violation_test_div = (
    physicality_violation_test_div_true + physicality_violation_test_div_false
)

NotImplementedError: 

In [416]:
true_object_table = report._convert_object_to_datafrane(true_object).to_html(escape=False, header=False)
tester_table = report._convert_objects_to_multiindex_dataframe(tester_objects).to_html(escape=False, header=False)

In [417]:
info = {"type of tomography": [qst.__class__.__name__],
           "n_rep": [iteration],
          "num_data": [num_data],
          }
condition_df = pd.DataFrame(info).T
condition_table = condition_df.to_html(escape=False, header=False)

In [426]:
table_css = """
table{
  border-collapse: collapse;
  border-spacing: 0;
}

table tr{
  border-bottom: solid 1px #666;
}

table th{
  text-align: right;
  background-color: #666;
  color: #fff;
  border-bottom: solid 1px #fff;
  border-right: solid 1px #fff;
  font-size: 13px;
  width: 100px;
  padding-top: 3px;
  padding-right: 3px;
}

table td{
  text-align: right;
  font-size: 13px;
  padding-top: 3px;
  padding-right: 3px;
  width: 400px;
}
"""

In [427]:
report_html = f"""<html>
<head>
    <script src="https://cdn.plot.ly/plotly-latest.min.js"></script>
    
    <style type="text/css">
        <!--
             body {{color: #666666;}}
             {{table_contents_css}}
             table {{border: solid 1px #666666;}}
             h1 {{margin-top: 60px; border-top: 2px #dcdcdc solid; padding-top: 10px; font-size: 25px}}
             h2 {{font-size: 20px}}
             h3 {{color:#EB9348; font-size: 15px;}}
             .box{{
                 display: inline-block;
                 width: 400px;
                }}
            h4{{font-size: 15px; color: #618CBC; }}
            {table_css}
            pdftoc {{
    color: #666;
}}
pdftoc.pdftoclevel0 {{
    font-weight: bold;
    margin-top: 0.5em;
}}
pdftoc.pdftoclevel1 {{
    margin-left: 1em;
}}
pdftoc.pdftoclevel2 {{
    margin-left: 2em;
    font-style: italic;
}}
         -->
    </style>
<title>タイトル</title>
</head>
<body>
<div id="table_of_contents">
<h1>Table of Contents</h1>
<pdf:toc />
</div>
<h1>Experimental Condition</h1>
<div>
{condition_table}
</div>
<h2>True Object</h2>
<div>
{true_object_table}
</div>
<h2>Tester Objects</h2>
<div>
{tester_table}
</div><h1>MSE</h1>
<div>
{mse_div}
</div>
<h1>Physicality Violation Test</h1>
{physicality_violation_test_div}

<div>
end
</div>
</body>
</html>"""

In [428]:
with open("test.html", "w") as f:
    f.write(report_html)
    
_convert_html2pdf(report_html, "sample.pdf")

0

In [429]:
!open test.html

In [430]:
!open sample.pdf